<a href="https://colab.research.google.com/github/SoumyaTeotia/Cross-Domain-Multi-Intent-Identification-in-Low-Resource-Languages/blob/main/ICON23_Method_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip -q install langchain openai tiktoken chromadb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import datasets
import os
import openai
import pandas as pd
import json

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DataFrameLoader
from getpass import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass()
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import openai

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   response = openai.Embedding.create(
   model="text-embedding-ada-002",
   input=text
   )
   return response['data'][0]['embedding']

  #  return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']


In [ ]:
# Generate embeddings for the two text items
text1 = "This is the first text item."
text2 = "This is the second text item."

embedding1 = get_embedding(text1)
embedding2 = get_embedding(text2)

# Compute the cosine similarity between the two embedding vectors
cosine_similarity = embedding1.dot(embedding2) / (embedding1.magnitude * embedding2.magnitude)

# Print the cosine similarity
print(cosine_similarity)

AttributeError: ignored

In [ ]:
!pip install openai[embeddings]

In [ ]:
from openai.embeddings_utils import cosine_similarity

ModuleNotFoundError: ignored

:Run this if you do not have a db directory on colab, this loads the embeddings.
# Only run this once

In [ ]:
def get_dataframe(jsonl_file_path):
  # Define the path to your JSONL file
  # jsonl_file_path = "hi-IN.jsonl"
  # Initialize an empty list to store the JSON objects
  data = []
  # Open and read the JSONL file line by line
  with open(jsonl_file_path, "r") as jsonl_file:
      for line in jsonl_file:
          data.append(json.loads(line))
  # Create a DataFrame from the list of JSON objects
  df = pd.DataFrame(data)
  return df
df = get_dataframe("hi-IN.jsonl")

def pre_process(df):
  df['slot_labels'] = df['slot_method']
  df['domain'] = df['scenario']
  df['annot_utt'] = df['annot_utt']
  df_reduced = df[['utt', 'intent', 'slot_labels', 'domain', 'annot_utt']].copy()
  for index, row in df_reduced.iterrows():
      slot_method_list = row['slot_labels']
      slot_values = ', '.join(item['slot'] for item in slot_method_list)
      df_reduced.at[index, 'slot_labels'] = slot_values
  # Create a new DataFrame with the desired format
  new_df = df_reduced[['utt', 'intent', 'slot_labels', 'domain', 'annot_utt']].copy()
  return new_df

new_df = pre_process(df)

def split_df(new_df):
  loader = DataFrameLoader(new_df, page_content_column="utt")
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
  texts = text_splitter.split_documents(documents)
  return texts
texts = split_df(new_df)

def store_embeddings(texts):
  persist_directory = 'db'
  embedding = OpenAIEmbeddings()
  vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)

  # persist the db to disk
  vectordb.persist()
  vectordb = None

  # Now we can load the persisted database from disk, and use it as normal.
  vectordb = Chroma(persist_directory=persist_directory,
                    embedding_function=embedding)
  return vectordb
vectordb = store_embeddings(texts)

In [ ]:
def get_relevant_docs(res):
  docs = []
  for doc in res['source_documents']:
    diction = {}
    for x in doc:
      if x[0] == 'page_content':
        diction["page_content"] = x[1]
      if x[0] == 'metadata':
        diction["metadata"] = x[1]
    docs.append(diction)
  return docs

def get_context(doc):
    res = []
    for doc in doc:
      input_doc = doc['page_content']
      intent = doc['metadata']['intent']
      domain = doc['metadata']['domain']
      #slot_labels = doc['metadata']['slot_labels']

      prompt = "The user utterance - Input : {}.\n Output : The intent is {}, domain is {} ".format(input_doc, intent, domain)
      res.append(prompt)
    return res


def generate_prompt(user_query, context):
  prompt_template = """

  You are an AI, that is expert in Natural Language Understanding. You should be able to predict the multiple intents and domains in english for the given Bengali input.
  \n {question}
  \n using the below examples in hindi, where the user utterance in given as Input and the domain and intent are given as Output.
  {context_prompt_list}
  \n You would be able to capture multiple domains and intents for the user utterence.

  Give the output in a json format as follows, append multiple intents, domains and slots to the list below.
  Format:

  "intent": []
  "domain": []

  """.format(question = user_query, context_prompt_list = '.\n'.join(context))

  return prompt_template

def get_intent_label(user_query):
  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": user_query}
    ]
  )
  return json.loads(response["choices"][0]["message"]["content"])

In [ ]:
def final(no_of_docs, user_query):
  template = """ Answer only using the context provided, if you don't know the answer, simply state that you don't know.
  The query will be in Hindi, use the stored Bengali data to answer the query

  {context}

  Question: {question}
  """
  # Creating a prompt
  PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])
  # Create a qa chain
  qa_with_source = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=vectordb.as_retriever(search_kwargs={"k": no_of_docs}),
    chain_type_kwargs={"prompt": PROMPT,},
    return_source_documents=True
  )
  # Get intents for that user_query
  res = qa_with_source(user_query)
  # Get the similar documents
  relevant_docs = get_relevant_docs(res)
  # Generate context vector
  context = get_context(relevant_docs)
  # Combine input and context to get prompt template
  prompt_template = generate_prompt(user_query, context)
  # Print the prompt template
  print(prompt_template)
  response = get_intent_label(prompt_template)
  # Final response after prompting
  return "The response after prompting is:\n {}".format(response)


In [ ]:
user_query_1 = "সোমবার মহালয়া থাকায়, অনুগ্রহ করে সকাল 4 টার জন্য অ্যালার্ম সেট করুন এবং রেডিও চালু করুন এবং পরিবেশ তৈরি করতে ভলিউমটি সম্পূর্ণ করুন।  এছাড়াও, আপনি কি কিছু ভাল বাঙ্গালী খাবার এর পরামর্শ দিতে পারেন যা আমি রান্না করতে সাহায্য করতে পারি। ভালো কিছু  গান সাজেস্ট করুন।"
user_query_2="আপনি কি আমাকে প্যারিস যাওয়ার জন্য একটি ফ্লাইটের পরামর্শ দিতে পারেন যেহেতু আমি খুব ঘন ঘন ভ্রমণকারী এবং বিশ্বের বিভিন্ন স্থান ঘুরে দেখতে পছন্দ করি। এছাড়াও আইফেল টাওয়ারের আশেপাশে কিছু ভাল রেস্তোরাঁ এবং হোটেলের সাথে কিছু পর্যটন স্থানের পরামর্শ দিন যা আমি ঘুরে দেখতে পারি?"
user_query_3="অস্টিনে আজ খুব ঠান্ডা। দিবালোক সঞ্চয় ইতিমধ্যে এখানে শুরু হয়েছে। আজকেও তুষারপাত হতে পারে। আমরা অনেক বাংলা গান বাজিয়েছিলাম এবং ডান্ডিয়া রাতে নাচতাম।"
user_query_split = user_query_1.split("।")
k = [8 , 12 , 16 , 32]

In [ ]:
for k_values in k:
  output = final(k_values , user_query_2)
  print(k_values)
  print(user_query_2)
  print(output)



  You are an AI, that is expert in Natural Language Understanding. You should be able to predict the multiple intents and domains in english for the given Bengali input.
  
 আপনি কি আমাকে প্যারিস যাওয়ার জন্য একটি ফ্লাইটের পরামর্শ দিতে পারেন যেহেতু আমি খুব ঘন ঘন ভ্রমণকারী এবং বিশ্বের বিভিন্ন স্থান ঘুরে দেখতে পছন্দ করি। এছাড়াও আইফেল টাওয়ারের আশেপাশে কিছু ভাল রেস্তোরাঁ এবং হোটেলের সাথে কিছু পর্যটন স্থানের পরামর্শ দিন যা আমি ঘুরে দেখতে পারি?
  
 using the below examples in hindi, where the user utterance in given as Input and the domain and intent are given as Output.
  The user utterance - Input : मेरे आसपास में किन स्थानों से मैं साथ ले जाने के लिए खरीदारी कर सकता हूँ.
 Output : The intent is takeaway_order, domain is takeaway .
The user utterance - Input : मैं स्थानीय पर्यटक के रूप में कहां से खरीदारी कर सकता हूं.
 Output : The intent is recommendation_locations, domain is recommendation .
The user utterance - Input : मैं लखनऊ के आसपास के सिनेमाघरों में कौन सी बाहुबली फिल्म देख सकत

In [ ]:
for users in user_query_1:
  for k_values in k:
    output = final(k_values , users)
    print(k_values)
    print(users)
    print(output)



  You are an AI, that is expert in Natural Language Understanding. You should be able to predict the multiple intents and domains in english for the given Bengali input.
  
 স
  
 using the below examples in hindi, where the user utterance in given as Input and the domain and intent are given as Output.
  The user utterance - Input : एस.
 Output : The intent is play_music, domain is play .
The user utterance - Input : सफ़ेद.
 Output : The intent is iot_hue_lightchange, domain is iot .
The user utterance - Input : सात सौ.
 Output : The intent is datetime_convert, domain is datetime .
The user utterance - Input : साथिया.
 Output : The intent is play_music, domain is play .
The user utterance - Input : सुखद.
 Output : The intent is cooking_recipe, domain is cooking .
The user utterance - Input : सागरतट.
 Output : The intent is qa_factoid, domain is qa .
The user utterance - Input : शतरंज.
 Output : The intent is play_game, domain is play .
The user utterance - Input : शाकाहार.
 Output :

JSONDecodeError: ignored

In [ ]:
# Get similarity score with actual
calculate_cosine_similarity(output, expected)

Save output to csv

In [ ]:
import csv

returned_string = output
csv_file = 'output.csv'

# Open the CSV file in write mode and create a CSV writer
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write each character as a separate row
    for i in output_list:
        writer.writerow([i])

print(f'Saved the string in "{csv_file}" as separate rows in a CSV file.')


Saved the string in "output.csv" as separate rows in a CSV file.
